In [4]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

Data read garyo

In [5]:
with open("data/anna.txt",'r') as f:
    text=f.read()

#### Tokenize

In [6]:
char = tuple(set(text))
int2char = dict(enumerate(char))
char2int = {ch:ii for ii,ch in enumerate(char)}
encoded = np.array([char2int[ch] for ch in text])
encoded.shape
len(char)

83

#### Preprocessing

In [7]:
def get_batches(enc_char,n_seq,n_steps):
    #batch_char(number of char in batch) = n_seq(i.e.batch_size)*n_steps(i.e. len of each batch)
    batch_char = n_seq * n_steps#say N*M
    #e.g. if we have batchsize= 5 and len of each batch = 4 then we will have first input as 5 equal sized batch with value 4
    total_no_batch = len(enc_char)// batch_char#// int dinxa
    #total char kati ho teslai divide by euta batch ko char garesi ayo
    total_char_tokeep = total_no_batch * batch_char
    #dim = k*N*M
    #thikka batch garauna milne gari char line 
    enc_char = enc_char[:total_char_tokeep]
    
    split_char = enc_char.reshape((n_seq,-1))
    #split char will be now dim = (N,M*K)
    for i in range(0,split_char.shape[1],n_steps): #split_char[1]=M*K i.e. total no of batch(k) and M= no_of_step in each batch
        x = split_char[:,i:i+n_steps]
        
        y = np.zeros_like(x)
        
        try:
            y[:,:-1],y[:,-1] = x[:,1:],x[:,i+n_steps]
            
        except IndexError:
            y[:,:-1],y[:,-1] = x[:,1:],x[:,0]
        
        yield x,y
            
            
        

    

In [8]:
def one_hot_encode(arr,n_labels):
    one_hot = np.zeros((np.multiply(*arr.shape),n_labels),dtype = np.float32)
    #yesle zero matrix banauxa of size (array ko size haru multiplication or flatten can also be done) 
    #jasari vaheni array lai vector ma convert garera tesko size*n_label ko matrix hunxa
    
    one_hot[np.arange(one_hot.shape[0]),arr.flatten()]=1
    #one_hot zero matrix hunxa 
    
    one_hot = one_hot.reshape((*arr.shape,n_labels))
    return one_hot

In [6]:
##batches = get_batches(encoded,10,50)
#harek sentence ma 10 ota batch tyo batch 50 step ko
##x,y = next(batches)

In [7]:
##print("x:",x[:,:10])
##print("y:",y[:,:10])


x: [[28 48 32 82 41 17  4 49 20 76]
 [49 32 53 49 37  1 41 49 45  1]
 [73 30 37 31 76 76 33 75 17 72]
 [37 49 77  2  4 30 37 45 49 48]
 [49 30 41 49 30 72 18 49 72 30]
 [49 26 41 49  6 32 72 76  1 37]
 [48 17 37 49 36  1 53 17 49  7]
 [62 49 56  2 41 49 37  1  6 49]
 [41 49 30 72 37 79 41 31 49 21]
 [49 72 32 30 77 49 41  1 49 48]]
y: [[48 32 82 41 17  4 49 20 76 76]
 [32 53 49 37  1 41 49 45  1 30]
 [30 37 31 76 76 33 75 17 72 18]
 [49 77  2  4 30 37 45 49 48 30]
 [30 41 49 30 72 18 49 72 30  4]
 [26 41 49  6 32 72 76  1 37 24]
 [17 37 49 36  1 53 17 49  7  1]
 [49 56  2 41 49 37  1  6 49 72]
 [49 30 72 37 79 41 31 49 21 48]
 [72 32 30 77 49 41  1 49 48 17]]


In [8]:
#x.shape

(10, 50)

In [9]:
#np.multiply(*x.shape),len(char)

(500, 83)

In [10]:
#a=np.zeros((x.flatten().size,len(char)),dtype=np.float32)
#print(a)
#print(a.shape)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
(500, 83)


In [11]:
#print(x.flatten())

[ 3 60 52  2 74 26 23 49 56 55 55 55 14 52  2  2 53 49 16 52  5 43 19 43
 26 32 49 52 23 26 49 52 19 19 49 52 19 43 69 26 34 49 26 66 26 23 53 49
 54 50 49 52  5 49 50 57 74 49 82 57 43 50 82 49 74 57 49 32 74 52 53 39
 61 49 52 50 32 71 26 23 26 17 49 27 50 50 52 39 49 32  5 43 19 43 50 82
 39 49 36 54 66 43 50 38 55 55 61 45 26 32 39 49 43 74 80 32 49 32 26 74
 74 19 26 17 38 49 44 60 26 49  2 23 43 30 26 49 43 32 49  5 52 82 50 43
 16 43 30 26 50 74 50 49 17 54 23 43 50 82 49 60 43 32 49 30 57 50 66 26
 23 32 52 74 43 57 50 49 71 43 74 60 49 60 43 32 55 36 23 57 74 60 26 23
 49 71 52 32 49 74 60 43 49 43 74 49 43 32 39 49 32 43 23 29 61 49 32 52
 43 17 49 74 60 26 49 57 19 17 49  5 52 50 39 49 82 26 74 74 43 50 82 49
 54  2 39 49 52 50 17 55 30 23 49 21 74 49 71 52 32 55 57 50 19 53 49 71
 60 26 50 49 74 60 26 49 32 52  5 26 49 26 66 26 50 43 50 82 49 60 26 49
 30 52  5 26 49 74 57 49 74 60 26 43 60 26 50 49 30 57  5 26 49 16 57 23
 49  5 26 39 61 49 32 60 26 49 32 52 43 17 39 49 52

In [24]:
##b= np.zeros((2,5),dtype=np.float32)
#y= np.array([3,4])
#b[np.arange(2),y]=1
#b[0,3] ma sure choti 1 ani b[1,4] ma arko choti 1

In [25]:
#y

array([3, 4])

In [26]:
#b

array([[0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 1.]], dtype=float32)

##### Model


In [9]:
class CharRNN(nn.Module):
    def __init__(self,tokens,n_hidden=256,num_layer=2,dropout_prob=0.2):
        super(CharRNN,self).__init__()
        self.dropout_prob = dropout_prob
        self.n_hidden = n_hidden
        self.num_layer = num_layer
        
        self.chars = tokens
        self.char2int = {ch:ii for ii, ch in enumerate(self.chars)}
        
        self.lstm = nn.LSTM(len(self.chars),n_hidden,num_layer,dropout=dropout_prob,batch_first=True)
        #n_hidden = number of feature in hidden state
        #num_layer = kati ota Lstm cell stacked up vaera baseko xa
        #batch_first = True means i/p and o/p are given
        
        self.dropout = nn.Dropout(p=dropout_prob)
        
        self.linear = nn.Linear(n_hidden,len(self.chars))
        
        self.init_weights()
        
    def forward(self,x,hc):
        out,(h,c) = self.lstm(x,hc)
        
        out = self.dropout(out)
        
        out = out.contiguous().view(-1,self.n_hidden)
        #contigous is required to stack up multiple LSTM cells as we have 2 LSTM cells defined
        
        out = self.linear(out)
        
        return out , (h,c)
    
    def init_weights(self):
        #it is done to create initial state of all zeros 
        initrange = 0.1
        
        self.linear.bias.data.fill_(0)
        #set bias tensor to zero
        
        self.linear.weight.data.uniform_(-1,1)
        
    def init_hidden(self,n_seq):
        weight = next(self.parameters()).data
        return (weight.new(self.num_layer, n_seq, self.n_hidden).zero_(),
                weight.new(self.num_layer, n_seq, self.n_hidden).zero_())
        

In [ ]:
#for param in net.parameters():
#    print(param.name,param.size())

#### At init_hidden :
When we create model defining a class with some layers. Each layer has some parameters associated with it. And once we instanciate our network with net=Model(). We can access those parameters as above.

Internally these parameters are in iterable mode as parameters() and we can iterate over using next().

next(self.parameters()).data will choose one of the parameters of the model. Which one exactly it chooses doesn't matter and it varies randomly depending on where the last pointer location was for the iterable over the parameters() container. Let's say it chooses the parameter fc from the model. When it chooses this, the device information and the data type of fc gets copied to the variable weight along with the data. The data isn't relevant for us, but the device information and the data type are important.

new() to create a new variable which is an exact copy of the weight variable and using the inplace function zero_() we get rid of the useless data from the weight variable and set the tensor to zero.

So, the basic reason why weight is needed, is so that it can act as a dummy variable that contains information about the device chosen, which we can then utilize to create a new tensor that will be thus automatically on the same device as the rest of the model. Thus essentially it is a trick to write code that will work whether you are training the model on GPU or using CPU.



#### Train Function

In [24]:
def train(net,data,epochs=10,n_seq=10,n_steps=50,lr=0.001,clip=5,val_frac=0.1,cuda=False,print_every=10):
    ''' Training a network 
    
        Arguments
        ---------
        
        net: CharRNN network
        data: text data to train the network
        epochs: Number of epochs to train
        n_seqs: Number of mini-sequences per mini-batch, aka batch size
        n_steps: Number of character steps per mini-batch
        lr: learning rate
        clip: gradient clipping
        val_frac: Fraction of data to hold out for validation
        cuda: Train with CUDA on a GPU
        print_every: Number of steps for printing training and validation loss
    
    '''
    net.train()
    #to specific training mode such that dropouts and other essential function for training is on
    opt = torch.optim.Adam(net.parameters(),lr=lr)
    criterion = nn.CrossEntropyLoss()
    
    val_idx = int(len(data)*(1-val_frac))
    #0.1 is fraction then val_idx will be from 0.9 fraction of total len
    
    dat,val_data = data[:val_idx],data[val_idx:]
    
    if cuda:
        net.cuda()
        
    counter = 0
    
    n_chars = len(net.chars)
    
    for epoch in range(epochs):
        hidden = net.init_hidden(n_seq)
        
        for x,y in get_batches(data,n_seq,n_steps):
            #x=(10,50) y=(10,50)
            counter +=1
            x = one_hot_encode(x,n_chars)
            #one_hot encode garesi input (batchno.,n_seq,n_char) i.e.(10,50,83)
            #now convert into torch
            inputs,targets = torch.from_numpy(x),torch.from_numpy(y)
            
            if cuda:
                inputs,targets = inputs.cuda(), targets.cuda()
            # Creating new variables for the hidden state, otherwise
            # we'd backprop through the entire training history
            h= tuple([each.data for each in hidden])
            
            net.zero_grad()
            
            output, h = net.forward(inputs, h)
            loss = criterion(output, targets.view(n_seqs*n_steps))
            #input = (10,50,83)
            # output will be in (n_seqs*n_step , len(chars)) i.e.(500,83)  one encoded o/p
            #target is (500) values
            loss.backward()
            
            nn.utils.clip_grad_norm_(net.parameters(), clip)
            #for exploding gradient
            opt.step()
            
            if counter % print_every == 0:
                
                # Get validation loss
                val_h = self.init_hidden(n_seq)
                val_loss = []
                net.eval()
                for x,y in get_batches(val_data,n_seq,n_steps):
                    x = one_hot_encode(x,n_chars)
                    
                    x,y = torch.from_numpy(x), torch.from_numpy(y)
                    val_h = tuple([each.data for each in val_h])
                    out , val_h = self.forward(x,val_h)
                    
                    val_los = criterion(out,y)
                    
                    val_loss.append(val_loss.item())
                 
                net.train()    
                
                print("Epoch: {}/{}...".format(e+1, epochs),
                      "Step: {}...".format(counter),
                      "Loss: {:.4f}...".format(loss.item()),
                      "Val Loss: {:.4f}".format(np.mean(val_losses)))
            

In [10]:
net = CharRNN(char, n_hidden=512, num_layer=2)
net

CharRNN(
  (lstm): LSTM(83, 512, num_layers=2, batch_first=True, dropout=0.2)
  (dropout): Dropout(p=0.2, inplace=False)
  (linear): Linear(in_features=512, out_features=83, bias=True)
)

In [17]:
counter =0
hidden = net.init_hidden(10)
for x,y in get_batches(encoded,10,50):
            #x=(10,50) y=(10,50)
    counter +=1
    x = one_hot_encode(x,len(net.chars))
            #one_hot encode garesi input (batchno.,n_seq,n_char) i.e.(10,50,83)
            #now convert into torch
    inputs,targets = torch.from_numpy(x),torch.from_numpy(y)
    h= tuple([each.data for each in hidden])

In [16]:
inputs.shape

torch.Size([10, 50, 83])

In [23]:
hidden

(tensor([[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]],
 
         [[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]]]),
 tensor([[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]],
 
         [[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
       

In [24]:
out,hid=net.lstm(inputs,hidden)

In [25]:
out.shape

torch.Size([10, 50, 512])

In [26]:
hid[1].shape

torch.Size([2, 10, 512])

In [15]:
n_seqs, n_steps = 128, 100

# you may change cuda to True if you plan on using a GPU!
# also, if you do, please INCREASE the epochs to 25
train(net, encoded, epochs=1, n_seq=n_seqs, n_steps=n_steps, lr=0.001, cuda=False, print_every=10)

NameError: name 'val_fraction' is not defined

In [ ]:
# change the name, for saving multiple files
model_name = 'rnn_1_epoch.net'

checkpoint = {'n_hidden': net.n_hidden,
              'n_layers': net.n_layers,
              'state_dict': net.state_dict(),
              'tokens': net.chars}

with open(model_name, 'wb') as f:
    torch.save(checkpoint, f)

### Predict Function

In [ ]:
def predict(net,char,h=None,top_k=None):
    
     # tensor inputs
    x = np.array([[net.char2int[char]]])
    #encoded input liyo char ko
    
    x = one_hot_encode(x, len(net.chars))
    #one hot encode garyo tesko
    
    inputs = torch.from_numpy(x)
        
    if(train_on_gpu):
            
        inputs = inputs.cuda()
        
        # detach hidden state from history
    h = tuple([each.data for each in h])
        # get the output of the model
    out, h = net(inputs, h)

        # get the character probabilities
    p = F.softmax(out, dim=1).data
    if(train_on_gpu):
        p = p.cpu() # move to cpu
        
        # get top characters
    if top_k is None:
        top_ch = np.arange(len(net.chars))
    else:
        p, top_ch = p.topk(top_k)
        top_ch = top_ch.numpy().squeeze()
        
        # select the likely next character with some element of randomness
    p = p.numpy().squeeze()
    char = np.random.choice(top_ch, p=p/p.sum())
        
        # return the encoded value of the predicted char and the hidden state
    return net.int2char[char], h

In [5]:
import numpy as np
np.array([[1,2],
         [2,4]]).shape

(2, 2)

In [ ]:
def sample(net, size, prime='The', top_k=None):
        
    if(train_on_gpu):
        net.cuda()
    else:
        net.cpu()
    
    net.eval() # eval mode
    
    # First off, run through the prime characters
    chars = [ch for ch in prime]
    h = net.init_hidden(1)
    for ch in prime:
        char, h = predict(net, ch, h, top_k=top_k)

    chars.append(char)
    
    # Now pass in the previous character and get a new one
    for ii in range(size):
        char, h = predict(net, chars[-1], h, top_k=top_k)
        chars.append(char)

    return ''.join(chars)

In [1]:
a = [1,2,3]

In [2]:
c= []
for i in a:
    b = i**2
    c.append(b)

In [3]:
c

[1, 4, 9]